### Define jobname

In [ ]:
pdbid = '2jpz'
drug  = None
jobname = f'{pdbid}_{drug}' if not drug is None else pdbid
indexfile=f'{jobname}.simulate.ndx'
mdp_folder = "../../../smsl/opt/mdp"

### make a centered reference structure

In [ ]:
def rmpbc(filename,indexfile='index.ndx',center=0,output=0, postfix='c'):
    file = ".".join(filename.split('.')[:-1]).split('_')[0]
    extend = filename.split('.')[-1]
    cmd1 = f'echo "{center} {output}" | gmx trjconv -s {file}.tpr -f {filename} -o {file}_{postfix}.{extend} -pbc res -center -boxcenter zero -n {indexfile} -ndec 5 -ur compact'
    print("##========================command=line========================##")
    print(cmd1,sep="\n")
    print("##======================end=command=line======================##")
    !{cmd1}

In [ ]:
def grep_and_fit(filename,indexfile='index.ndx',center=0,output=0, postfix='rms'):
    file = ".".join(filename.split('.')[:-1]).split('_')[0]
    extend = filename.split('.')[-1]
    cmd1 = f'echo "{center} {output}" | gmx trjconv -s rms.tpr -f {filename} -o {file}_{postfix}.{extend} -n {indexfile} -fit rot+trans -ndec 5'
    print("##========================command=line========================##")
    print(cmd1,sep="\n")
    print("##======================end=command=line======================##") 
    !{cmd1}

In [ ]:
rmpbc("npt2.pdb", indexfile, center='DNA')

### make a tpr using centered reference structure

In [ ]:
!gmx grompp -f {mdp_folder}/min2.mdp -c npt2_c.pdb -r npt2_c.pdb -p topol.top -o rms.tpr

### Center protein for a solvated trajectory and put residues in the unit cell 
- Input and output are both solvated trajectories
- 1. make residues in the unit cell
- The sequence of commands here are used to generate the "process.solvated.traj" shell scripts for production runs

In [ ]:
rmpbc("heat.xtc", indexfile, center='DNA')
rmpbc("nvt1.xtc", indexfile, center='DNA')
rmpbc("npt1.xtc", indexfile, center='DNA')
rmpbc("npt2.xtc", indexfile, center='DNA')

### get a no-solvent reference structure

In [ ]:
!echo "Model"     | gmx editconf -f npt2_c.pdb -o npt2_rms.pdb -n {jobname}.simulate.ndx
!echo "non-Water" | gmx editconf -f npt2_c.pdb -o npt2_noh.pdb -n {jobname}.simulate.ndx

### Output non-water parts with best fit 
- The index file needs to be setup first
- The sequence of commands here are used to generate the "rms.traj" shell scripts for production runs

In [ ]:
grep_and_fit("heat_c.xtc", indexfile, center='DNA', output='Model')
grep_and_fit("nvt1_c.xtc", indexfile, center='DNA', output='Model')
grep_and_fit("npt1_c.xtc", indexfile, center='DNA', output='Model')
grep_and_fit("npt2_c.xtc", indexfile, center='DNA', output='Model')

In [ ]:
grep_and_fit("heat_c.xtc", indexfile, center='DNA', output='non-Water', postfix='noh')
grep_and_fit("nvt1_c.xtc", indexfile, center='DNA', output='non-Water', postfix='noh')
grep_and_fit("npt1_c.xtc", indexfile, center='DNA', output='non-Water', postfix='noh')
grep_and_fit("npt2_c.xtc", indexfile, center='DNA', output='non-Water', postfix='noh')

### Cat rms trajectories
- Use cat.traj to combine production run trajectories

In [ ]:
cmd = '''\
c
c
c
c
'''

!echo -e "{cmd}" | gmx trjcat -f heat_c.xtc nvt1_c.xtc npt1_c.xtc npt2_c.xtc -o equilibrium_c.xtc -settime

In [ ]:
cmd = '''\
c
c
c
c
'''

!echo -e "{cmd}" | gmx trjcat -f heat_noh.xtc nvt1_noh.xtc npt1_noh.xtc npt2_noh.xtc -o equilibrium_noh.xtc -settime

In [ ]:
cmd = '''\
c
c
c
c
'''

!echo -e "{cmd}" | gmx trjcat -f heat_rms.xtc nvt1_rms.xtc npt1_rms.xtc npt2_rms.xtc -o equilibrium_rms.xtc -settime

## Visualize
```bash
pymol rms.pdb heat_rms.xtc  
vmd -gro rms.pdb heat_rms.xtc  
```

### Move data to next folder

In [ ]:
!cp {indexfile} rms.tpr           ../2.simulation  ## for rmpbc
!cp npt2.pdb npt2.cpt *.top *.itp ../2.simulation  ## for mdrun

In [ ]:
!cp npt2_rms.pdb                  ../3.trajectory_data/{jobname}.npt2.pdb